# Dependency setup

## Package setup

In [65]:
# Import dependencies and libraries
from IPython.display import clear_output
import pandas as pd
from census import Census
from us import states
from dotenv import load_dotenv
import requests
import sys
import csv
import os
import json
import time

# Load development mode
try:
  dev = os.getenv("DEV_MODE")
  if dev: print("Development mode loaded successfully!")
except:
  print("Error loading development mode")
  print("Defaulting to production mode")
  dev = False

# Load environment variables and U.S. Census API key
if dev: print("Loading environment variables...")
load_dotenv()
if dev: print("Environment variables loaded successfully!")
try:
  if dev: print("Loading U.S. Census API key...")
  api_key = os.getenv("CENSUS_API_KEY")
  if dev: print("API key loaded successfully!")
except:
  print("Error loading U.S. Census API key")
  print("Please check your environment variables")
  

Development mode loaded successfully!
Loading environment variables...
Environment variables loaded successfully!
Loading U.S. Census API key...
API key loaded successfully!


## Google Colab setup

In [ ]:
# Install the required libraries for running on colab
try:
  %pip install prophet
except:
  print("Error installing libraries to colab instance")
finally:
  clear_output()
  print("Libraries successfully installed to colab instance")


# Census Code

In [ ]:
# Create an instance of the Census library
try:
  if dev: print("Creating an instance of the Census API...")
  c = Census(
      api_key
  )
  if dev: print("Census API instance created:")
  if dev: display(type(c))
  if dev: display(c)
except:
  print("Error creating an instance of the Census API")
  print("Please check your U.S. Census API key")


## Census API Aliases

In [ ]:
# Load census aliases
# TODO: Add google colab-specific code to load the census aliases
if dev: print("Loading census aliases...")
with open('data/census/census_aliases.json') as f:
    census_aliases = json.load(f)

# Display the census aliases
# display(census_aliases)

def printAliases():
  # Show the keys in the census aliases
  alias_count = 1
  print("Census aliases:")
  print("-"*30)
  for key, value in census_aliases.items():
      description = value["description"]
      column_name = value["variable"]
      title = value["text"]
      unit = value["unit"]
      print(f"{alias_count:03d} | {title.capitalize()}: {description}")
      print(f"    | Census column name: {column_name}")
      print(f"    | Unit: {unit}")
      print("-"*30)
      alias_count+=1

# Saving the reference of the standard output
original_stdout = sys.stdout 	

try:
  # Save the census aliases to a file
  if dev: print("Saving census aliases to file...")
  with open('census_aliases.txt', 'w') as f:
      sys.stdout = f
      printAliases()
      # Reset the standard output
      sys.stdout = original_stdout
  print("Census aliases saved to file successfully!")
except:
  print("Error saving census aliases to file")


## FIPS Codes

### Functions for obtaining county FIPS codes

In [ ]:
# Define funtions to get FIPS codes for US counties
# https://gist.github.com/cjwinchester/a8ff5dee9c07d161bdf4
def getCounties():
    "Function to return a dict of FIPS codes (keys) of U.S. counties (values)"
    d = {}
    r = requests.get("http://www2.census.gov/geo/docs/reference/codes/files/national_county.txt")
    reader = csv.reader(r.text.splitlines(), delimiter=',')    
    for line in reader:
        d[line[1] + line[2]] = line[3].replace(" County","")    
    return d

def getCountyAdj():
    "Return a list of dicts where each dict has a county FIPS code (key) and a list of FIPS codes of the adjacent counties, not including that county (value)"
    adj = requests.get("http://www2.census.gov/geo/docs/reference/county_adjacency.txt")
    adj_data = adj.text.encode("utf-8")
    reader = csv.reader(adj_data.splitlines(), delimiter='\t')
    ls = []
    d = {}
    countyfips = ""
    for row in reader:
        if row[1] and row[1] != "":
            if d:
                ls.append(d)
            d = {}
            countyfips = row[1]
            d[countyfips] = []
            "Grab the record on the same line"
            try:
                st = row[3]
                if st != countyfips:
                    d[countyfips].append(st)
            except:
                pass
        else:
            "Grab the rest of the records"
            if row[3] and row[3] != "":
                st = row[3]
                if st != countyfips:
                    d[countyfips].append(st)


### Obtaining state FIPS codes

In [ ]:
# Get the FIPS codes for Colorado
try:
  if dev: print("Getting FIPS codes for Colorado...")
  colorado_fips = states.CO.fips
  state_fips = colorado_fips
  print(f"Colorado FIPS code: {state_fips}")
except:
  print("Error getting FIPS codes for Colorado")


### Obtaining all US county FIPS codes

In [ ]:
# Store county FIPS codes in memory
try:
  if dev: print("Storing county FIPS codes in memory...")
  county_fips = getCounties()
  if dev: print("All County FIPS codes:")
  if dev: display(county_fips)
except:
  print("Error storing county FIPS codes in memory")
  

### Obtaining FIPS codes for all counties in Colorado

In [ ]:
# Filter out only Colorado county FIPS codes
try:
  if dev: print("Filtering out only Colorado county FIPS codes...")
  colorado_county_fips = {key: value for key, value in county_fips.items() if key.startswith(state_fips)}
  if dev: print("Colorado County FIPS codes:")
  if dev: display(colorado_county_fips)
  print(f"Number of Colorado counties: {len(colorado_county_fips)}")
except:
  print("Error filtering out only Colorado county FIPS codes")


### Obtaining FIPS code for a specific county in Colorado

In [ ]:
# Prompt for county to look up and retrieve fip
# county_name = input("Enter county name to look up the FIPS code: ")
try:
  if dev: print("Looking up the FIPS code for Denver county...")
  county_name = "Denver"
  state_county_fip = list(county_fips.keys()) [list(county_fips.values()).index(county_name)]
  county_fip = state_county_fip[2:]
  print(f"The FIPS code for {county_name} is {county_fip}")
except:
  print(f"Error looking up the FIPS code for {county_name} county")


## Census Data

### Getting Raw Census Data

In [ ]:
# Run Census Search to retrieve data on all zip codes
census_start_year = 2011
census_end_year = 2021
census_years = range(census_start_year, census_end_year + 1)
data_retrieval_output = sys.stdout

# Create a DataFrame to hold the census data with an empty row so we can append data to it
raw_census_df = pd.DataFrame({
  "NAME": "",
  "B19013_001E": "",
  "B19301_001E": "",
  "B23025_002E": "",
  "B23025_007E": "",
  "B17001_002E": "",
  "B01002_001E": "",
  "B01003_001E": "",
  "B25035_001E": "",
  "B25058_001E": "",
  "B25064_001E": "",
  "B25077_001E": "",
  "B25088_002E": "",
  "Year": ""
}, index=[0])

# Loop through the years and retrieve data for each year
try:
  for county_fip in colorado_county_fips.keys():
    for year in census_years:
      if dev: print(f"Retrieving data for {year} {colorado_county_fips[county_fip]} County...")
      year_data = c.acs5.state_county(
        (
          "NAME",
          "B19013_001E",
          "B19301_001E",
          "B23025_002E",
          "B23025_007E",
          "B17001_002E",
          "B01002_001E",
          "B01003_001E",
          "B25035_001E",
          "B25058_001E",
          "B25064_001E",
          "B25077_001E",
          "B25088_002E"
        ),
        colorado_fips,
        county_fip[2:],
        year=year
      )

      # Display the retrieved data
      if dev: print(f"Data retrieved for {year} {colorado_county_fips[county_fip]} County.")
      if dev: display(year_data)

      # Convert to DataFrame
      if dev: print("Converting data to DataFrame...")
      year_df = pd.DataFrame(year_data)

      # Add the data's year to a Year column
      if dev: print(f"Adding Year column to {year} {colorado_county_fips[county_fip]} County...")
      year_df["Year"] = year

      # Add the data to the combined DataFrame
      if dev: print("Adding data to the combined DataFrame...")
      raw_census_df = pd.concat([raw_census_df, year_df], ignore_index=True)

      # Wait for 5 seconds before making the next request
      if dev: print("Waiting for 5 seconds before making the next request...")
      if dev: print("-"*50)
      # TODO: Show percentage of data retrieval completion
      clear_output(wait=True)
      time.sleep(5)
    print(f"Data retrieval of {colorado_county_fips[county_fip]} County complete.")
  print(f"Data retrieval of all Colorado counties complete.")
except Exception as e:
  print("Error retrieving data from the U.S. Census API")
  print(e)


### Saving Raw Census Data to CSV

In [ ]:
# Save the raw census data to a CSV file
try:
  if dev: print("Saving raw census data to a CSV file...")
  raw_census_df.to_csv("data/census/raw_census_data.csv", index=False)
  print("Raw census data saved to file successfully!")
except:
  print("Error saving raw census data to file")
  

### Loading Raw Census Data from CSV

In [ ]:
# Load the raw census data from the CSV file
try:
  if dev: print("Loading raw census data from the CSV file...")
  raw_census_df = pd.read_csv("data/census/raw_census_data.csv")
  if dev: display(raw_census_df.head())
  print("Raw census data loaded from file successfully!")
except:
  print("Error loading raw census data from file, please make sure the file exists.")


### Cleaning Census Data

#### Dropping rows with missing values

In [ ]:
# Drop the empty row from the combined DataFrame
try:
  if dev: print("Dropping the empty row from the combined DataFrame...")
  raw_census_df.dropna(inplace=True)
  if dev: print("Combined DataFrame:")
  display(raw_census_df.head())
except:
  print("Error dropping empty row from the combined DataFrame")


#### Index fixing and Sorting

In [ ]:
# Copy the DataFrame to a new DataFrame
if dev: print("Copying the DataFrame to a new DataFrame...")
census_df = raw_census_df.copy()
# Reset the index of the DataFrame
if dev: print("Resetting the index of the DataFrame...")
census_df.reset_index(inplace=True, drop=True)
# Sort the DataFrame by Name and Year
if dev: print("Sorting the DataFrame by Name and Year...")
census_df.sort_values(by=["NAME", "Year"], inplace=True)
# Display the DataFrame
if dev: print("New DataFrame:")
if dev: display(census_df.head())


#### Column fixing

In [ ]:
# Convert the Year column to an integer
try:
  if dev: print("Converting the Year column to an integer...")
  census_df["Year"] = census_df["Year"].astype(int)
  if dev: print("Year column converted to an integer successfully!")
except:
  print("Error converting the Year column to an integer")


In [ ]:
# Column renaming
if dev: print("Renaming columns...")
census_df.rename(
  columns = {
    "NAME": "County",
    "B19013_001E": "Median Household Income",
    "B19301_001E": "Per Capita Income",
    "B23025_002E": "Number of Employed (16+)",
    "B23025_007E": "Number of Unemployed (16+)",
    "B17001_002E": "Poverty Count",
    "B01002_001E": "Median Age",
    "B01003_001E": "Total Population",
    "B25035_001E": "Average Year Home Built",
    "B25058_001E": "Median Contract Rent",
    "B25064_001E": "Average Cost of Rent and Utilities",
    "B25077_001E": "Average Value of Owner-occupied Home",
    "B25088_002E": "Average Monthly Housing Expense with Mortgage"
  },
  inplace=True
)

# Add a Poverty Rate column (Poverty Count / Population)
if dev: print("Adding a Poverty Rate column...")
census_df["Poverty Rate"] = 100 * census_df["Poverty Count"].astype(int) / census_df["Population"].astype(int)


In [ ]:
# Configure the final DataFrame
if dev: print("Configuring the DataFrame...")
census_df = census_df[
  [
    "County",
    "Year",
    "Median Household Income",
    "Per Capita Income",
    "Number of Employed (16+)",
    "Number of Unemployed (16+)",
    "Poverty Count",
    "Median Age",
    "Total Population",
    "Average Year Home Built",
    "Median Contract Rent",
    "Average Cost of Rent and Utilities",
    "Average Value of Owner-occupied Home",
    "Average Monthly Housing Expense with Mortgage"
  ]
]

# Display DataFrame length and sample data
if dev: print(f"Number of rows in the DataFrame: {len(census_df)}")
if dev: print("Combined county DataFrame preview:")
if dev: display(census_df.head())


#### Saving Cleaned Census Data to CSV

In [ ]:
# Save the final census data to a CSV file
try:
  if dev: print("Saving final census data to a CSV file...")
  census_df.to_csv("data/census/clean_census_data.csv", index=False)
  print("Final census data saved to file successfully!")
except:
  print("Error saving final census data to file")


#### Loading Cleaned Census Data from CSV

In [ ]:
# Load the final census data from the CSV file
try:
  if dev: print("Loading final census data from the CSV file...")
  census_df = pd.read_csv("data/census/clean_census_data.csv")
  if dev: display(census_df.head())
  print("Final census data loaded from file successfully!")
except:
  print("Error loading final census data from file, please make sure the file exists.")


#### Displaying Cleaned Census Data

In [ ]:
# Display the first and last 30 rows of the DataFrame
print("First 30 rows of the DataFrame:")
display(census_df.head(30))

print("Last 30 rows of the DataFrame:")
display(census_df.tail(30))


# School Code

In [73]:
# Import School data into a DataFrame
if dev: print("Loading combined school data...")
raw_school_data_df = pd.read_excel("data/schools/combined_2011-2021.xlsx")

print("School data loaded successfully!")


Loading combined school data...
School data loaded successfully!


In [74]:
# Copy the DataFrame to a new DataFrame
if dev: print("Copying the DataFrame to a new DataFrame...")
school_data_df = raw_school_data_df.copy()
# Sort the DataFrame by Name and Year
if dev: print("Sorting the DataFrame by County and Year...")
school_data_df.sort_values(by=["County", "Year"], inplace=True)
# Reset the index of the DataFrame
school_data_df.reset_index(inplace=True, drop=True)
# Display the DataFrame
if dev: print("New DataFrame:")
if dev: display(school_data_df.head())


Copying the DataFrame to a new DataFrame...
Sorting the DataFrame by County and Year...
New DataFrame:


,County,Year,"Shelters, transitional housing, awaiting foster care",Doubled-up,Unsheltered,Hotels/Motels,TOTAL
0,ADAMS,2011,199,3963,58,176,4396
1,ADAMS,2012,208,3470,45,148,3871
2,ADAMS,2013,166,3511,58,139,3874
3,ADAMS,2014,210,3778,91,206,4285
4,ADAMS,2015,227,3434,73,183,3917


In [75]:
# Display the first and last 30 rows of the DataFrame
print("School data:")
display(school_data_df.head(30))
display(school_data_df.tail(30))


School data:


,County,Year,"Shelters, transitional housing, awaiting foster care",Doubled-up,Unsheltered,Hotels/Motels,TOTAL
0,ADAMS,2011,199,3963,58,176,4396
1,ADAMS,2012,208,3470,45,148,3871
2,ADAMS,2013,166,3511,58,139,3874
3,ADAMS,2014,210,3778,91,206,4285
4,ADAMS,2015,227,3434,73,183,3917
5,ADAMS,2016,221,3195,67,125,3608
6,ADAMS,2017,200,3562,50,180,3992
7,ADAMS,2018,173,3335,52,198,3758
8,ADAMS,2019,170,2741,63,191,3165
9,ADAMS,2020,132,1869,52,149,2202


,County,Year,"Shelters, transitional housing, awaiting foster care",Doubled-up,Unsheltered,Hotels/Motels,TOTAL
674,WASHINGTON,2014,0,10,0,0,10
675,WASHINGTON,2015,0,17,1,0,18
676,WASHINGTON,2016,0,21,1,0,22
677,WASHINGTON,2017,0,23,0,0,23
678,WASHINGTON,2018,2,18,5,0,25
679,WASHINGTON,2019,1,21,8,0,30
680,WASHINGTON,2020,1,24,16,1,42
681,WASHINGTON,2021,0,22,5,1,28
682,WELD,2011,54,639,37,40,770
683,WELD,2012,68,826,14,45,953


# HIC Data

In [66]:
# Import HIC data into a DataFrame
if dev: print("Loading HIC data...")
raw_hic_data_df = pd.read_excel("data/hic/HIC_CO_2007-2023.xlsx")

print("HIC data loaded successfully!")


Loading HIC data...
HIC data loaded successfully!


In [67]:
# Display the first and last 30 rows of the DataFrame
print("HIC data:")
display(raw_hic_data_df.head(30))
display(raw_hic_data_df.tail(30))


HIC data:


,Year,State,"Total Year-Round Beds (ES, TH, SH)","Total Non-DV Year-Round Beds (ES, TH, SH)","Total HMIS Year-Round Beds (ES, TH, SH)","HMIS Participation Rate for Year-Round Beds (ES, TH, SH)",Total Year-Round Beds (ES),Total Year-Round Beds (TH),Total Year-Round Beds (SH),"Total Units for Households with Children (ES, TH, SH)",...,Total Year-Round Beds (OPH),Total Non-DV Year-Round Beds (OPH),Total HMIS Year-Round Beds (OPH),HMIS Participation Rate for Year-Round Beds (OPH),Total Units for Households with Children (OPH),Total Beds for Households with Children (OPH),Total Beds for Households without Children (OPH),Total Beds for Households with only Children (OPH),Dedicated Veteran Beds (OPH),Dedicated Youth Beds (OPH)
0,2007,CO,6414,5735,3597,0.6270,2780,3634,1235,3584,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2008,CO,6522,5712,4374,0.7660,2744,3753,25,1217,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2009,CO,6363,5532,4707,0.8510,2729,3609,25,1206,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2010,CO,6420,5625,4930,0.8760,2591,3804,25,1223,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2011,CO,6361,5620,4944,0.8800,2619,3717,25,1262,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2012,CO,6536,5960,5444,0.9130,2495,4016,25,1193,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2013,CO,7744,7028,6022,0.8570,3153,4358,208,25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2014,CO,8466,7625,5861,0.6923,3753,4688,25,1295,...,820.0,2271.0,0.0,10.0000,10.0,0.0,0.0,1.0,10.0,0.0
8,2015,CO,7653,6834,5168,0.6753,3263,4365,25,1136,...,853.0,2481.0,0.0,10.0000,10.0,0.0,0.0,1.0,10.0,0.0
9,2016,CO,7656,6834,4310,0.5630,3541,4048,67,1068,...,1089.0,2886.0,0.0,54.0000,54.0,54.0,1.0,0.0,0.0,54.0


,Year,State,"Total Year-Round Beds (ES, TH, SH)","Total Non-DV Year-Round Beds (ES, TH, SH)","Total HMIS Year-Round Beds (ES, TH, SH)","HMIS Participation Rate for Year-Round Beds (ES, TH, SH)",Total Year-Round Beds (ES),Total Year-Round Beds (TH),Total Year-Round Beds (SH),"Total Units for Households with Children (ES, TH, SH)",...,Total Year-Round Beds (OPH),Total Non-DV Year-Round Beds (OPH),Total HMIS Year-Round Beds (OPH),HMIS Participation Rate for Year-Round Beds (OPH),Total Units for Households with Children (OPH),Total Beds for Households with Children (OPH),Total Beds for Households without Children (OPH),Total Beds for Households with only Children (OPH),Dedicated Veteran Beds (OPH),Dedicated Youth Beds (OPH)
0,2007,CO,6414,5735,3597,0.6270,2780,3634,1235,3584,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2008,CO,6522,5712,4374,0.7660,2744,3753,25,1217,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2009,CO,6363,5532,4707,0.8510,2729,3609,25,1206,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2010,CO,6420,5625,4930,0.8760,2591,3804,25,1223,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2011,CO,6361,5620,4944,0.8800,2619,3717,25,1262,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2012,CO,6536,5960,5444,0.9130,2495,4016,25,1193,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2013,CO,7744,7028,6022,0.8570,3153,4358,208,25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2014,CO,8466,7625,5861,0.6923,3753,4688,25,1295,...,820.0,2271.0,0.0,10.0000,10.0,0.0,0.0,1.0,10.0,0.0
8,2015,CO,7653,6834,5168,0.6753,3263,4365,25,1136,...,853.0,2481.0,0.0,10.0000,10.0,0.0,0.0,1.0,10.0,0.0
9,2016,CO,7656,6834,4310,0.5630,3541,4048,67,1068,...,1089.0,2886.0,0.0,54.0000,54.0,54.0,1.0,0.0,0.0,54.0


# PIT Data

In [68]:
# Import PIT data into a DataFrame
if dev: print("Loading PIT data...")
raw_pit_data_df = pd.read_excel("data/pit/PIT_CO_2007-2023.xlsx")

print("PIT data loaded successfully!")


Loading PIT data...
PIT data loaded successfully!


In [69]:
# Display the first and last 30 rows of the DataFrame
print("PIT data:")
display(raw_pit_data_df.head(30))
display(raw_pit_data_df.tail(30))


PIT data:


,Year,State,Number of CoCs,Overall Homeless,Overall Homeless - Under 18,Overall Homeless - Age 18 to 24,Overall Homeless - Over 24,Overall Homeless - Female,Overall Homeless - Male,Overall Homeless - Transgender,...,Unnamed: 634,Unnamed: 635,Unnamed: 636,Unnamed: 637,Unnamed: 638,Unnamed: 639,Unnamed: 640,Unnamed: 641,Unnamed: 642,Unnamed: 643
0,2007,CO,3,14225,3630,3341,6971,7254,6844,2158,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2008,CO,3,14747,3492,3385,6877,7870,6260,2104,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2009,CO,3,15268,4895,4136,9031,6237,7401,3083,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2010,CO,3,15482,4895,4136,0,9031,6451,7574,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2011,CO,3,15116,2979,3142,25,6146,8970,4546,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2012,CO,3,16768,3502,3733,25,7260,9508,5101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2013,CO,3,9754,2710,1066,5978,4025,785,419,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2014,CO,3,10028,2835,1004,6189,3701,5284,38,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2015,CO,3,9953,2528,808,6617,3923,6012,18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2016,CO,3,10550,2327,983,7240,4103,6393,54,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,Year,State,Number of CoCs,Overall Homeless,Overall Homeless - Under 18,Overall Homeless - Age 18 to 24,Overall Homeless - Over 24,Overall Homeless - Female,Overall Homeless - Male,Overall Homeless - Transgender,...,Unnamed: 634,Unnamed: 635,Unnamed: 636,Unnamed: 637,Unnamed: 638,Unnamed: 639,Unnamed: 640,Unnamed: 641,Unnamed: 642,Unnamed: 643
0,2007,CO,3,14225,3630,3341,6971,7254,6844,2158,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2008,CO,3,14747,3492,3385,6877,7870,6260,2104,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2009,CO,3,15268,4895,4136,9031,6237,7401,3083,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2010,CO,3,15482,4895,4136,0,9031,6451,7574,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2011,CO,3,15116,2979,3142,25,6146,8970,4546,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2012,CO,3,16768,3502,3733,25,7260,9508,5101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2013,CO,3,9754,2710,1066,5978,4025,785,419,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2014,CO,3,10028,2835,1004,6189,3701,5284,38,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2015,CO,3,9953,2528,808,6617,3923,6012,18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2016,CO,3,10550,2327,983,7240,4103,6393,54,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# SNAP Data

In [70]:
# Import SNAP data into a DataFrame
if dev: print("Loading SNAP data...")
raw_snap_data_df = pd.read_csv("data/snap/SNAP_CO_2010-2022.csv")

print("SNAP data loaded successfully!")


Loading SNAP data...
SNAP data loaded successfully!


In [71]:
# Display the first and last 30 rows of the DataFrame
print("SNAP data:")
display(raw_snap_data_df.head(30))
display(raw_snap_data_df.tail(30))


SNAP data:


,DATE,BRCO08M647NCEN_20231214
0,1/1/10,"395,580"
1,2/1/10,"398,862"
2,3/1/10,"407,731"
3,4/1/10,"411,245"
4,5/1/10,"413,805"
5,6/1/10,"417,989"
6,7/1/10,"419,737"
7,8/1/10,"424,314"
8,9/1/10,"424,878"
9,10/1/10,"428,941"


,DATE,BRCO08M647NCEN_20231214
120,1/1/20,"434,142"
121,2/1/20,"431,397"
122,3/1/20,"439,355"
123,4/1/20,"507,074"
124,5/1/20,"524,084"
125,6/1/20,"539,331"
126,7/1/20,"551,936"
127,8/1/20,"534,175"
128,9/1/20,"523,098"
129,10/1/20,"506,102"


# SSS Data

In [ ]:
# Import SSS data into a DataFrame
if dev: print("Loading SSS data...")
raw_sss_data_df = pd.read_excel("data/sss/SSS_CO_2011-2022.xlsx")

print("SSS data loaded successfully!")


In [ ]:
# Display the first and last 30 rows of the DataFrame
print("SSS data:")
display(raw_sss_data_df.head(30))
display(raw_sss_data_df.tail(30))
